In [ ]:
import json
from kreis_loader import get_kreise

kreise = get_kreise(returnGeometry=True)

with open("../kreise.json", "w") as json_file:
    json.dump(kreise, json_file)

In [ ]:
from kreis_loader import get_envelope
from stations_loader import stations_find, filter_stations
from map_drawer import DrawMap as dm
import json

# Open the file for reading
with open('../kreise.json', 'r') as json_file:
    kreise = json.load(json_file)
kreise

# Create an instance of DrawMap
dm = dm()


flensburg = kreise[0]
flens_map = dm.plot_regions(flensburg)

polygon = flensburg["geometry"]
envelope = get_envelope(polygon)
flens_map = dm.add_envelope(envelope)

stations = stations_find(geometry = envelope)
#flens_map = dm.add_stations(stations)

stations_filtered = filter_stations(polygon, stations)
flens_map = dm.add_stations(stations_filtered)

flens_map

In [ ]:
from kreis_loader import get_kreise, get_envelope
from stations_loader import stations_find, filter_stations
from map_drawer import DrawMap as dm
import json

kreise = get_kreise(returnGeometry=True)

# Create an instance of DrawMap
dm = dm()

for kreis in kreise:
    station_map = dm.plot_regions(kreis)

    polygon = kreis.get("geometry", None)  # Using get() to avoid KeyError
    if polygon is not None:
        # Compute the envelope and add it to the kreis
        envelope = get_envelope(polygon)
        kreis['envelope'] = envelope
    
    stations = stations_find(geometry = envelope)
    kreis['stations'] = stations

    stations_filtered = filter_stations(polygon, stations)
    kreis['stations_filtered'] = stations_filtered
    station_map = dm.add_stations(kreis["stations_filtered"])

with open("../kreise.json", "w") as json_file:
    json.dump(kreise, json_file)

station_map